In [2]:
import numpy as np
import bilby

In [3]:
duration = 64.
sampling_frequency = 256.

outdir = 'outdir'
label = 'eccentric_GW140914'
bilby.core.utils.setup_logger(outdir=outdir, label=label)

# Set up a random seed for result reproducibility.
np.random.seed(150914)

injection_parameters = dict(
    mass_1=35., mass_2=30., eccentricity=0.1, luminosity_distance=440.,
    theta_jn=0.4, psi=0.1, phase=1.2, geocent_time=1180002601.0, ra=45, dec=5.73)

waveform_arguments = dict(waveform_approximant='EccentricFD',
                          reference_frequency=10., minimum_frequency=10.)

# Create the waveform_generator using the LAL eccentric black hole no spins
# source function
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_eccentric_binary_black_hole_no_spins,
    parameters=injection_parameters, waveform_arguments=waveform_arguments)


# Setting up three interferometers (LIGO-Hanford (H1), LIGO-Livingston (L1), and
# Virgo (V1)) at their design sensitivities. The maximum frequency is set just
# prior to the point at which the waveform model terminates. This is to avoid
# any biases introduced from using a sharply terminating waveform model.
minimum_frequency = 10.
maximum_frequency = 128.

ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
for ifo in ifos:
    ifo.minimum_frequency = minimum_frequency
    ifo.maximum_frequency = maximum_frequency
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 3)
ifos.inject_signal(waveform_generator=waveform_generator,
                   parameters=injection_parameters)

# Now we set up the priors on each of the binary parameters.
priors = bilby.core.prior.PriorDict()
priors["mass_1"] = bilby.core.prior.Uniform(
    name='mass_1', minimum=5, maximum=60, unit='$M_{\\odot}$')
priors["mass_2"] = bilby.core.prior.Uniform(
    name='mass_2', minimum=5, maximum=60, unit='$M_{\\odot}$')
priors["eccentricity"] = bilby.core.prior.LogUniform(
    name='eccentricity', latex_label='$e$', minimum=1e-4, maximum=0.4)
priors["luminosity_distance"] = bilby.gw.prior.UniformComovingVolume(
    name='luminosity_distance', minimum=1e2, maximum=2e3)
priors["dec"] = bilby.core.prior.Cosine(name='dec')
priors["ra"] = bilby.core.prior.Uniform(
    name='ra', minimum=0, maximum=2 * np.pi)
priors["theta_jn"] = bilby.core.prior.Sine(name='theta_jn')
priors["psi"] = bilby.core.prior.Uniform(name='psi', minimum=0, maximum=np.pi)
priors["phase"] = bilby.core.prior.Uniform(
    name='phase', minimum=0, maximum=2 * np.pi)
priors["geocent_time"] = bilby.core.prior.Uniform(
    1180002600.9, 1180002601.1, name='geocent_time', unit='s')

# Initialising the likelihood function.
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=ifos, waveform_generator=waveform_generator)



14:12 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_eccentric_binary_black_hole_no_spins
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
/home/hemantakumar.phurailatpam/anaconda3/envs/bilby/lib/python3.7/site-packages/bilby/gw/detector/psd.py:356: RuntimeWarning: invalid value encountered in multiply
  frequency_domain_strain = self.__power_spectral_density_interpolated(frequencies) ** 0.5 * white_noise
14:12 bilby INFO    : Injected signal in H1:
14:12 bilby INFO    :   optimal SNR = 66.39
14:12 bilby INFO    :   matched filter SNR = 67.49-0.58j
14:12 bilby INFO    :   mass_1 = 35.0
14:12 bilby INFO    :   mass_2 = 30.0
14:12 bilby INFO    :   eccentricity = 0.1
14:12 bilby INFO    :   luminosity_distance = 440.0
14:12 bilby INFO    :   theta_jn = 0.4
14:12 bilby INFO    :   psi = 0.1
14:12 bilby INFO    :   phase = 1.2
14:12 bilby INFO    :   geocent_time 

In [5]:
# Now we run sampler (PyMultiNest in our case).
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='pymultinest', npoints=1000,
    injection_parameters=injection_parameters, outdir=outdir, label=label)

14:15 bilby INFO    : Running for label 'eccentric_GW140914', output will be saved to 'outdir'
14:15 bilby INFO    : Using lal version 6.21.0
14:15 bilby INFO    : Using lal git version Branch: None;Tag: lal-v6.21.0;Id: 37caf89936b637993c30b3076fe4b853311873ee;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
14:15 bilby INFO    : Using lalsimulation version 1.10.0
14:15 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsimulation-v1.10.0;Id: 5bfe091197594d55cf634f8a1c004eb9dc9b6459;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed



ERROR:   Could not load MultiNest library "libmultinest.so"
ERROR:   You have to build it first,
ERROR:   and point the LD_LIBRARY_PATH environment variable to it!
ERROR:   manual: http://johannesbuchner.github.com/PyMultiNest/install.html


ERROR:   Could not load MultiNest library: libmultinest.so
ERROR:   You have to build MultiNest,
ERROR:   and point the LD_LIBRARY_PATH environment variable to it!
ERROR:   manual: http://johannesbuchner.github.com/PyMultiNest/install.html

problem: libmultinest.so: cannot open shared object file: No such file or directory


SamplerNotInstalledError: Sampler pymultinest is not installed on this system